In [32]:
import duckdb as ddb

# Installing and loading the HTTPFS extension to handle file access via HTTP/S
ddb.sql("INSTALL httpfs; LOAD httpfs")

con = ddb.connect("../air_quality.db")

# Creating a schema named 'raw' if it does not already exist
con.execute("CREATE schema IF NOT EXISTS raw")

# Setting up credentials for accessing S3 (empty here, presumably placeholders for real credentials)
con.sql("""
    SET s3_access_key_id='';
    SET s3_secret_access_key='';
    SET s3_region='';
""")

# Creating a table 'air_quality_data' within the 'raw' schema if it does not already exist
# The table is designed to store air quality data with columns for location, time, coordinates, parameters, and more
con.execute("""
    CREATE TABLE IF NOT EXISTS raw.air_quality_data (
        location_id BIGINT,
        sensors_id BIGINT,
        "location" VARCHAR,
        "datetime" TIMESTAMP,
        lat DOUBLE,
        lon DOUBLE,
        "parameter" VARCHAR,
        units VARCHAR,
        "value" DOUBLE,
        "month" VARCHAR,
        "year" BIGINT,
        ingestion_datetime TIMESTAMP
    );
""")

# Inserting data into the 'raw.air_quality_data' table from a CSV file stored in an S3 bucket
# Data is read from a compressed file using the 'read_csv' function
con.execute("""
    INSERT INTO raw.air_quality_data
    SELECT 
        location_id, 
        sensors_id, 
        "location", 
        "datetime", 
        lat, 
        lon, 
        "parameter", 
        units, 
        "value",
        "month", 
        "year",
        current_timestamp AS ingestion_datetime
    FROM read_csv('s3://openaq-data-archive/records/csv.gz/locationid=2027500/year=2024/month=01/*.csv.gz');
""")

con.close()
